<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/gamszej/system_rekomendacyjny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from datetime import datetime
import time

In [ ]:
dataGrocery = pd.read_csv('GroceryStoreDataSet.csv', names=["products"])
dataGrocery.head(10)

In [ ]:
dataGroceryStructured = dataGrocery['products'].apply(lambda x: x.split(','))
uniqueItems = set([item for sublist in dataGroceryStructured for item in sublist])
sorted(uniqueItems)

dataGrocery['products'] = dataGrocery['products'].str.replace('COCK', 'COKE').str.replace('SUGER', 'SUGAR')
dataGroceryStructured = dataGrocery['products'].apply(lambda x: x.split(','))

transaction_encoder = TransactionEncoder()
dataGroceryEncoded = transaction_encoder.fit(dataGroceryStructured).transform(dataGroceryStructured)
dataGroceryEncoded = pd.DataFrame(dataGroceryEncoded, columns=transaction_encoder.columns_).astype(int)

In [ ]:
#Para produktów o największym lift i  największej ufności

frequent_itemsets = apriori(dataGroceryEncoded, min_support=0.15, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
max_confidence_rule = rules[rules['confidence'] == rules['confidence'].max()].head(1)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.1)
max_lift_rule = rules[rules['lift'] == rules['lift'].max()].head(1)

In [ ]:
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

In [ ]:
print("Missing values in ratings:\n", ratings.isnull().sum())

In [ ]:
print("Missing values in movies:\n", movies.isnull().sum())

In [ ]:
print("Ratings: ", ratings['rating'].nunique(), sorted(ratings['rating'].unique()))
ratings['rating'].value_counts()

In [ ]:
print("Amount of ratings: ", ratings.shape[0])
print("Movies creted: ", movies['movieId'].nunique(),", Unique Titles", movies['title'].nunique())
print("Movies revied: ", movies['movieId'].nunique())
print("Amount of users: ", ratings['userId'].nunique(),", Highest Id", ratings['userId'].max())
print("Average amount of ratings per user: ", round(ratings.shape[0]/ratings['userId'].nunique(),2))
print("Average amount of ratings per movie: ", round(ratings.shape[0]/ratings['movieId'].nunique(),2))

In [ ]:
titles = list(movies['title'])
duplicates = set([x for x in titles if titles.count(x) > 1])

print("Duplicated titles:")
for duplicate in duplicates:
  print("  -",duplicate)

movies[movies['title'].isin(duplicates)]

In [ ]:
movie_titles_dict = dict(zip(movies['movieId'], movies['title']))

In [ ]:
av = ratings.shape[0]/ratings['movieId'].nunique()
avargeOfAll = ratings['rating'].sum()/ratings.shape[0]
print("Average rating: ",av, ", Average rating of all movies: ",avargeOfAll)

In [ ]:
def bayesian_avg(ratings):
    bayesian_avg = (av * avargeOfAll + ratings.sum()) / (av + ratings.count())
    return bayesian_avg

bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
movie_stats = pd.merge(movies, bayesian_avg_ratings, on='movieId')

print(movie_stats.head())

In [ ]:
def create_sparse(df):
  userId = df['userId'].nunique()
  movieId = df['movieId'].nunique()
  user_mapper = dict(zip(np.unique(df["userId"]), list(range(userId))))
  movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(movieId))))
  user_inv_mapper = dict(zip(list(range(userId)), np.unique(df["userId"])))
  movie_inv_mapper = dict(zip(list(range(movieId)), np.unique(df["movieId"])))
  user_index = [user_mapper[i] for i in df['userId']]
  item_index = [movie_mapper[i] for i in df['movieId']]
  X = csr_matrix((df["rating"], (user_index, item_index)), shape=(userId,movieId))
  return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

  X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_sparse(ratings)

In [ ]:
def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    X = X.T
    neighbour_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1,-1)
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [ ]:
n_total = X.shape[0]*X.shape[1]
n_ratings = X.nnz
sparsity = n_ratings/n_total
sparsity_p = round(sparsity*100,2)
print(f"Matrix sparsity: {sparsity_p}%")
n_ratings_per_user = X.getnnz(axis=1)

In [ ]:
n_ratings_per_user = X.getnnz(axis=1)
print("Amount of ratings per user:", n_ratings_per_user)

In [ ]:
toyStoryID = movies[movies['title'].str.contains('Toy Story \(')]['movieId'][0]
similarMovies = find_similar_movies(toyStoryID, X, movie_mapper, movie_inv_mapper, k=10, metric='cosine')
print(f"Most similar movies (Id):  {toyStoryID}: {similarMovies}")
print(f"\nMost similar movies (Title):  {list(movies[movies['movieId']==toyStoryID]['title'])}:")
movies[movies['movieId'].isin(similarMovies)]['title']

In [ ]:
toyStoryID = movies[movies['title'].str.contains('Toy Story \(')]['movieId'][0]
toyStoryID

In [ ]:
movies['genres'] = movies['genres'].str.lower()
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)', expand=False)

In [ ]:
genreLists = list(movies['genres'].apply(lambda x: x.lower().split('|')))
genreList = []

for genres in genreLists:
  genreLists.extend(genres)

print("Set of unique genres (lowercase):", set(genreList))
genre_df = pd.DataFrame(genreList)
genre_df.value_counts()

In [ ]:
#Podaj liczbę filmów dla dzieci
children_movies_count = movies("Number of children movies:", children_movies_count)

ratings['rating_year'] = pd.to_datetime(ratings['timestamp'], unit='s').dt.year

In [ ]:
#Pokaż rozkład ocen filmów z 1995
import matplotlib.pyplot as plt
ratingsJoin = ratings.merge(movies, on='movieId', how='inner')
ratings_1995 = ratingsJoin[ratingsJoin['year'] == '1995']
ratings_distribution_1995 = ratings_1995['rating'].value_counts().sort_index()
print("Distribution of 1995 film ratings:")
print(ratings_distribution_1995)
plt.figure(figsize=(10, 6))
ratings_distribution_1995.plot(kind='bar')
plt.title('Distribution of 1995 film ratings')
plt.xlabel('Rate')
plt.ylabel('Number of appearances')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
#Podaj średnią ocen wszystkich filmów akcji oraz 3 filmy najwyżej oceniane
actionMovies = ratingsJoin[ratingsJoin['genres'].str.contains('action', case=False)]

average_rating_action = actionMovies['rating'].mean()
print(f"Average ratings for action movies: {average_rating_action}")

top_three = actionMovies.groupby('title')['rating'].mean().sort_values(ascending=False).head(3)
print(f"\nThe three highest rated action films: {top_three}")

In [ ]:
!pip install surprise

In [ ]:
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.prediction_algorithms import SVD, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import Reader
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

min_rating = ratings.rating.min()
max_rating = ratings.rating.max()
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings[['userId','movieId' , 'rating']], reader)
trainset, testset = train_test_split(data, test_size = 0.25)

In [ ]:
#svd
start_time_svd_train = time.time()
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)
end_time_svd_train = time.time()

In [ ]:
param_grid = {
    'n_factors': [1,2,5,10, 50]
}

start_time_svd = time.time()
gsSVD = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gsSVD.fit(data)
end_time_svd = time.time()
gsSVD.best_params

cross_validate(SVD(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
#knn basic
lgo = KNNBasic()
start_time_knn_train = time.time()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)
end_time_knn_train = time.time()

In [ ]:
param_grid = {
    'k': [1, 2, 5, 10, 20, 30, 40, 50]
}
start_time_knn = time.time()

gsKNN = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gsKNN.fit(data)
end_time_knn = time.time()
gsKNN.best_params

cross_validate(KNNBasic(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
#knn baseline
algo = KNNBaseline()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

In [ ]:
#porównanie
bestKNN = gs_knn.best_estimator['rmse']
bestSVD = gs_svd.best_estimator['rmse']

start_time_knn_cross = time.time()
resultsKNN = cross_validate(bestKNN, data, measures=['rmse'], cv=5, verbose=True)
end_time_knn_cross = time.time()

start_time_svd_cross = time.time()
resultsSVD = cross_validate(bestSVD, data, measures=['rmse'], cv=5, verbose=True)
end_time_svd_cross = time.time()

print(f"\nBest KNN RMSE: {gs_knn.best_score['rmse']}")
print(f"Best SVD RMSE: {gs_svd.best_score['rmse']}")
print(f"\nBest KNN Parameters: {gs_knn.best_params['rmse']}")
print(f"Best SVD Parameters: {gs_svd.best_params['rmse']}")
print(f"\nHyperparameters KNN Training Time: {end_time_knn - start_time_knn} seconds")
print(f"Hyperparameters SVD Training Time: {end_time_svd - start_time_svd} seconds")
print(f"\nCros KNN Training Time: {end_time_knn - start_time_knn} seconds")
print(f"Cros SVD Training Time: {end_time_svd_cross - start_time_svd_cross} seconds")
print(f"\nTrain SVD Training Time: {end_time_svd_train - start_time_svd_train} seconds")
print(f"Train KNN Training Time: {end_time_knn_train - start_time_knn_train} seconds")
print(f"\nKNN Cross-validation Results: {resultsKNN}")
print(f"SVD Cross-validation Results: {resultsSVD}")